In [ ]:
import os
import pandas as pd
from SignalProcessor import SignalProcessor

pd.options.plotting.backend = 'plotly'

repo_dir = '/home/cam/Documents/database_tools/'
os.chdir(repo_dir)

worker = SignalProcessor(
    files=['physionet.org/files/mimic3wdb/1.0/30/3000480/3000480_0018'],
    win_len=1024,
    fs=125,
)

windows, snr, hr = worker.run(
    sim1=0.6,
    sim2=0.9,
    snr_t=20,
    hr_diff=1/6,
    f0_low=0.667,
    f0_high=3.0
)
len(windows)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from Preprocessing.SignalLevelFiltering import get_snr

i = 0
x, y = windows[i][0], windows[i][1]

scaler = MinMaxScaler(feature_range=(60, 120))
x = scaler.fit_transform(x.reshape(-1, 1)).reshape(-1,)

df = pd.DataFrame(
    dict(
        pleth=x,
        abp=y,
    )
)

df.plot()

In [ ]:
import numpy as np

x = np.array([0, 1, 2, 3])
np.logical_and(x >= 1, x <= 2)

In [ ]:
get_snr(y, low=0.5, high=8, fs=100)

In [ ]:
from Preprocessing.SignalLevelFiltering import get_snr
import heartpy as hp

x = hp.load_exampledata()[0]
pd.Series(x).plot()